In [22]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from ibm_watson_machine_learning import APIClient
import json

In [23]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='thtVtg0WoZuBwG83os1x4QmlmKfCqFObEj0OEYsWSLXD',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'loanprediction-donotdelete-pr-ehjawyfoacc37o'
object_key = 'train.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,0,0,0,4583,0,112,360,1,0,0
1,1,1,0,0,0,1820,1719,100,360,1,2,1
2,1,1,0,0,0,3033,1459,95,360,1,2,1
3,0,1,0,0,0,3667,1459,144,360,1,1,1
4,1,1,3,0,0,3466,3428,150,360,1,0,1


In [24]:
train,test = train_test_split(df_data_1, test_size=0.33, random_state=42)

In [25]:
x=df_data_1.drop(["Loan_Status"],axis=1)
y=df_data_1.Loan_Status
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [26]:
def randomForest(x_train, x_test, y_train, y_test):
    rf = RandomForestClassifier() 
    rf.fit(x_train,y_train) 
    yPred = rf.predict(x_test) 
    print('***RandomForestClassifier***') 
    print('Confusion matrix')
    print(confusion_matrix(y_test,yPred))
    print('Classification report')
    print(classification_report(y_test,yPred))
    print("score")
    print(rf.score(x_test,y_test))

In [27]:
randomForest(x_train, x_test, y_train, y_test)

***RandomForestClassifier***
Confusion matrix
[[48 10]
 [ 4 31]]
Classification report
              precision    recall  f1-score   support

           0       0.92      0.83      0.87        58
           1       0.76      0.89      0.82        35

    accuracy                           0.85        93
   macro avg       0.84      0.86      0.84        93
weighted avg       0.86      0.85      0.85        93

score
0.8494623655913979


In [33]:
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
yPred = rf.predict(x_test)
f1_score(yPred,y_test, average='weighted')
cv = cross_val_score(rf,x,y,cv=5)
np.mean(cv)

0.8205002337540905

In [34]:
!pip install -U ibm-watson-machine-learning

In [38]:
wml_credentials={
    "apikey":"xnDDhAdOjxcO_dQeVyKQnMumIGziTBtDIQjhQvUUSHVh",
    "url":"https://eu-de.ml.cloud.ibm.com"
}

In [41]:
wml_client = APIClient(wml_credentials)

In [42]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ---------------------  ------------------------
ID                                    NAME                   CREATED
57c1758e-acbf-4e1b-80bd-a5b120791b3e  loan_deployment_space  2022-11-19T07:39:28.885Z
0694da06-8889-4086-b1f7-354034a88896  new_page               2022-10-14T23:58:52.110Z
------------------------------------  ---------------------  ------------------------


In [43]:
SPACE_ID= "57c1758e-acbf-4e1b-80bd-a5b120791b3e"

In [44]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [45]:
import sklearn
sklearn.__version__

'1.0.2'

In [46]:
MODEL_NAME = 'DeployModel'
DEPLOYMENT_NAME = 'loan_deploy'
DEMO_MODEL = rf

In [47]:
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [48]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [49]:
model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=x_train, 
    training_target=y_train
)

In [50]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'Loan_Status',
  'schemas': {'input': [{'fields': [{'name': 'Gender', 'type': 'int64'},
      {'name': 'Married', 'type': 'int64'},
      {'name': 'Dependents', 'type': 'int64'},
      {'name': 'Education', 'type': 'int64'},
      {'name': 'Self_Employed', 'type': 'int64'},
      {'name': 'ApplicantIncome', 'type': 'int64'},
      {'name': 'CoapplicantIncome', 'type': 'int64'},
      {'name': 'LoanAmount', 'type': 'int64'},
      {'name': 'Loan_Amount_Term', 'type': 'int64'},
      {'name': 'Credit_History', 'type': 'int64'},
      {'name': 'Property_Area', 'type': 'int64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-19T07:44:25.644Z',
  'id': '8071f140-471b-4a76-a175-62501efdecce',
  'modified_at': '2022-11-19T07:44:28.873Z',
  'name': 

In [51]:
model_id = wml_client.repository.get_model_id(model_details)
model_id

'8071f140-471b-4a76-a175-62501efdecce'

In [52]:
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [53]:
deployment = wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)




#######################################################################################

Synchronous deployment creation for uid: '8071f140-471b-4a76-a175-62501efdecce' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='767f401b-2a43-435e-8c2f-3920659bd30d'
------------------------------------------------------------------------------------------------


